In [220]:
from wandb.integration.keras import WandbMetricsLogger
from mlp_explainer import explainer, metrics
from scikeras.wrappers import KerasClassifier
from pgmpy.utils import get_example_model
from tensorflow import keras
import mlp_explainer
import numpy as np
import sklearn
import wandb
import copy

In [221]:
wandb.login()

True

In [234]:
from ucimlrepo import fetch_ucirepo 
  
car_evaluation = fetch_ucirepo(id = 19) 

X = car_evaluation.data.features.to_numpy()
y = car_evaluation.data.targets.to_numpy()

In [235]:
def build_model(input_shape, classes, n_layers = 5, n_nodes_l = 2, n_nodes_u = 10, optimizer = 'adam', learning_rate = 0.001):
    
    model = keras.Sequential()
    model.add(keras.Input(shape = (input_shape,)))
    for i in range(n_layers):
        model.add(keras.layers.Dense(np.random.choice([i for i in range(n_nodes_l, n_nodes_u)]), activation = np.random.choice(['relu', 'sigmoid'])))
    model.add(keras.layers.Dense(classes, activation = 'softmax'))

    if optimizer == 'sgd':
        optim = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        optim = keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [236]:
n_layers = 5
n_nodes_l = 5
n_nodes_u = 50
optimizer = 'adam'
learning_rate = 0.01
epochs = 25
batch_size = 64

In [237]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, test_size = 0.1, random_state = 42, shuffle = True
)

In [238]:
feature_encoder = sklearn.preprocessing.OrdinalEncoder()
feature_encoder.fit(X_train)

target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(y_train.squeeze())

LabelEncoder()

In [239]:
X_train_proc = feature_encoder.transform(X_train)
y_train_proc = target_encoder.transform(y_train)

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [240]:
n_classes = len(np.unique(y))
input_shape = len(X[0])

In [241]:
wandb.init(entity = "computer-vision-wits", project = "MLP-E", name = "")
    
model = build_model(
    input_shape,
    n_classes,
    n_layers = n_layers, 
    n_nodes_l = n_nodes_l,
    n_nodes_u = n_nodes_u, 
    optimizer = optimizer, 
    learning_rate = learning_rate
)
    
config = wandb.config
config.n_layers = n_layers
config.n_nodes_l = n_nodes_l
config.n_nodes_u = n_nodes_u
config.optimizer = optimizer
config.epochs = epochs
config.batch_size = batch_size
config.learning_rate = learning_rate
    
model.fit(X_train_proc, y_train_proc, epochs = epochs, batch_size = batch_size, validation_split = 0.2, callbacks = [WandbMetricsLogger()])
model.summary()

wandb.finish()

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/wandb/analytics/sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


Epoch 1/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6833 - loss: 0.8869 - val_accuracy: 0.6656 - val_loss: 0.8855
Epoch 2/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7162 - loss: 0.7322 - val_accuracy: 0.6656 - val_loss: 0.8122
Epoch 3/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7162 - loss: 0.6654 - val_accuracy: 0.6656 - val_loss: 0.7734
Epoch 4/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7066 - loss: 0.6345 - val_accuracy: 0.6656 - val_loss: 0.7055
Epoch 5/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7235 - loss: 0.5728 - val_accuracy: 0.7138 - val_loss: 0.6384
Epoch 6/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7629 - loss: 0.5223 - val_accuracy: 0.7460 - val_loss: 0.5731
Epoch 7/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8151 - loss: 0.4371 - val_accuracy: 0.7846 - val_loss: 0.4592
Epoch 8/25
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8344 - loss: 0.3741 - val_accuracy: 0.8039 - val_loss:

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_132 (Dense)               │ (None, 26)             │           182 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_133 (Dense)               │ (None, 15)             │           405 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_134 (Dense)               │ (None, 45)             │           720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_135 (Dense)               │ (None, 16)             │           736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_136 (Dense)               │ (None, 15)             │           255 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_137 (Dense)               │ (None, 4)              │            64 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,088 (27.69 KB)

 Trainable params: 2,362 (9.23 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,726 (18.46 KB)

epoch/accuracy,▁▂▂▂▂▃▄▅▅▅▆▆▇▇▆▆▇▇▇██████
epoch/epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▆▅▅▄▄▃▃▃▂▂▂▃▂▂▂▂▁▁▁▁▁▁
epoch/val_accuracy,▁▁▁▁▂▃▄▄▅▅▆▆▆▅▃▇▇▇▆█▇█▇██
epoch/val_loss,█▇▇▆▆▅▄▄▄▄▂▂▃▄▆▂▂▂▂▁▂▁▂▁▁
epoch/accuracy,0.97508
epoch/epoch,24
epoch/learning_rate,0.01
epoch/loss,0.08084
epoch/val_accuracy,0.9582


In [242]:
X_test_proc = feature_encoder.transform(X_test)
y_test_proc = target_encoder.transform(y_test)

y_pred = model(X_test_proc)
y_pred = np.argmax(y_pred, axis = 1)
sklearn.metrics.accuracy_score(y_pred, y_test_proc)

/home/vmuser/anaconda3/envs/venv/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0.930635838150289

In [311]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules

class Explainer:
    def __init__(self, model):
        
        self.model = model
        self.data = None
        self.patterns = []
        
    def __data_generation(self, x: np.ndarray, X_train: np.ndarray, preproc, x_cols: list, y_col: str, n: int = 10, prob: float = 0.5):
        
        X = []
        Y = []
        
        proc_x = preproc.transform(x.reshape(1, -1))
        y = self.model.predict(proc_x, verbose = 0)
        y = y.reshape(len(y[0]),)
        
        for i in range(n):
            
            idx = np.random.randint(len(X_train))
            x_prime = X_train[idx]
            
            mask = np.random.rand(len(x_prime)) < prob
            x_prime = np.where(mask, x, x_prime)

            proc_x_prime = preproc.transform(x_prime.reshape(1, -1))
            y_prime = self.model.predict(proc_x_prime, verbose = 0)
            y_prime = y_prime.reshape(len(y_prime[0]),)
            
            y_prime = y_prime.argmax() != y.argmax()
            x_prime = x_prime != x
            
            X.append(x_prime.astype(int))
            Y.append(y_prime.astype(int))
        
        self.data = pd.DataFrame(X, columns = x_cols)
        self.data[y_col] = Y

    def fp_growth(self, data, class_):

        if class_ == 0:
            data = 1 - data
        data = data.astype(bool)

        self.patterns.append(fpgrowth(data, min_support = 0.3, use_colnames = True))

    def __relevance_rank(self):

        self.patterns[0] = self.patterns[0].rename(columns = {'support': 'support_stable'})
        self.patterns[1] = self.patterns[1].rename(columns = {'support': 'support_unstable'})
        
        patterns_merged = pd.merge(
            self.patterns[0],
            self.patterns[1],
            on = 'itemsets',
            how = 'outer'
        )
        
        patterns_merged = patterns_merged.fillna(0)
        
        support_0 = patterns_merged['support_stable']
        support_1 = patterns_merged['support_unstable']
        
        denominator = support_0 + support_1
        patterns_merged['Harmonic Mean'] = (2 * support_0 * support_1 / denominator).mask(denominator == 0, 0)
        
        ranked = patterns_merged[['itemsets', 'support_stable', 'support_unstable', 'Harmonic Mean']].sort_values(
            by = 'Harmonic Mean', 
            ascending = False
        )
        
        return ranked[ranked['Harmonic Mean'] > 0.0]

    def explain(self, x, X_train, preproc, x_cols, y_col, n, prob) -> None:
        
        x = np.asarray(x)
        
        self.__data_generation(x, X_train, preproc, x_cols, y_col, n, prob)
        classes = np.unique(self.data[y_col].values)
        
        for class_ in classes:
            data = self.data.loc[self.data[y_col] == class_]
            data = data.drop(y_col, axis = 1)
            self.fp_growth(data, class_)

        return self.__relevance_rank()

In [312]:
x_cols = car_evaluation.data.features.columns
y_col = car_evaluation.data.targets.columns[0]

In [346]:
for _ in range(10):
    explainer = Explainer(model)
    x = X_test[np.random.randint(len(X_test))]
    rank = explainer.explain(x, X_train, feature_encoder, x_cols, y_col, 1000, 0.25)
    print(f'{rank}\n')

             itemsets  support_stable  support_unstable  Harmonic Mean
2            (safety)        0.717765          0.980132       0.828678
5            (buying)        0.457020          0.609272       0.522276
3             (doors)        0.451289          0.539735       0.491566
0          (lug_boot)        0.481375          0.390728       0.431341
9    (buying, safety)        0.335244          0.589404       0.427393
1             (maint)        0.372493          0.493377       0.424497
7     (doors, safety)        0.332378          0.519868       0.405500
6  (lug_boot, safety)        0.362464          0.390728       0.376066
4           (persons)        0.416905          0.301325       0.349815

               itemsets  support_stable  support_unstable  Harmonic Mean
0             (persons)        0.632944          1.000000       0.775218
1               (maint)        0.521401          0.790393       0.628318
2               (doors)        0.487678          0.567686       0.5246

safety 
 persons 
 buying 
 maint 
 lug_boot 
 doors
